Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

## Problem Description

random notes: 

- GOAL: maximizing the value and minimizing the weight of objects carried by the <NUM_KNAPSACKS> knapsacks

- We should respect the contraints on the weight, which defines how much weights a knapsack can carry.

- a dimension defines a contraint on a napsack.
Ex. [x, y, z] -> 3 constraints on backpack 1.

- The solution is represented by a 2D matrix (NUM_KNAPSACKS, NUM_ITEMS). Each row represents a knapsack and for each row we have a list of carried items where its length = NUM_ITEMS. Each element of the list is either a 1 if the knapsack carries the item, or is 0 if it doesn't carry the item.

- the fitness of the function is defined by overall value carried by all the knapsacks combined (sum of single total value of each knapsack)

In [1]:
import numpy as np
from random import randint
import math

In [2]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [3]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [4]:
CONSTRAINTS

array([[157, 132],
       [298, 124],
       [143, 263]])

In [5]:
# A random solution - starting point
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=np.bool
)

### UTILS

In [6]:
def get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS):
    return {
        "NUM_KNAPSACKS": NUM_KNAPSACKS,
        "NUM_ITEMS": NUM_ITEMS,
        "NUM_DIMENSIONS": NUM_DIMENSIONS,
        "VALUES": VALUES,
        "WEIGHTS": WEIGHTS,
        "CONSTRAINTS": CONSTRAINTS,
    }

In [7]:
def generate_random_solution(num_knapsacks, num_items):
    """
    generate a pure random solution. In this case is possible to start
    from a starting point where you have items that are carried by more than 1 knapsack.
    """
    
    solution = np.array(
        [np.random.random(num_items) < 0.5 for _ in range(num_knapsacks)], dtype=np.int8
    )

    return solution

In [8]:
def generate_clean_random_solution(num_knapsacks, num_items):
    """
    generate a random solution that satisfy the first constraint
    (that is each item is either not carried or carried from exactly 1 item).
    """

    # generate a 2D matrix (num_knapsacks, num_items) full of 0s.
    solution = np.zeros((num_knapsacks, num_items), dtype=int)

    # iterate over the columns
    for item_idx in range(num_items):

        # generate random number:
        # if 0 the item is not carried
        # if 1 the item is carried by 1 knapsack
        carried = randint(0, 1)
    
        if carried == 0:
            continue
        
        # generate a random index that identifies the knapsack that carries the item
        id_knapsack = randint(0, num_knapsacks - 1)

        # update the solution
        solution[id_knapsack, item_idx] = 1
        
    return solution

In [9]:
def fitness(solution, problem):
    """
    Calculates the fitness of a solution.
    """

    # check the basic constraint: an item cannot be carried by more then 1 knapsack
    # if the sum of the items in the same column >= 1 it means that
    # at leats 2 knapsacks are carrying the item, so a fitness of 0 is returned
    if np.any(np.sum(solution, axis=0) > 1):
        return 0

    values = problem["VALUES"]
    weights = problem["WEIGHTS"]
    constraints = problem["CONSTRAINTS"]

    # sum all the values for each row based on the items that each knapsack carries
    total_value = np.sum(solution * values)
    # get the total weights for each knapsack
    knapsack_loads = solution @ weights

    # Check if any constraint is violated
    if np.any(knapsack_loads > constraints):
        return 0 
    else:
        return total_value

### ALGORITHMS

In [10]:
def hill_climbing(problem, max_iterations=10000):
    """
    Performs Hill Climbing.
    """

    # get the variables from the given problem
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]

    # generate a random solution (starting point)
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)

    # calculate the fitness for the current_solution
    current_fitness = fitness(current_solution, problem)
    print(f"Starting Hill Climbing with initial fitness: {current_fitness}")

    for i in range(max_iterations):
        # create a neighbor solution by making one small change
        neighbor_solution = current_solution.copy()
        
        # pick a random item to move
        item_to_move = np.random.randint(num_items)
        # pick a new knapsack for it (-1 means removing it)
        new_knapsack = np.random.randint(-1, num_knapsacks)

        # remove the item from wherever it is
        neighbor_solution[:, item_to_move] = 0
        # place it in its new knapsack (if not being removed)
        if new_knapsack != -1:
            neighbor_solution[new_knapsack, item_to_move] = 1

        # evaluate the neighbor by calculating its fitness
        neighbor_fitness = fitness(neighbor_solution, problem)

        # if the neighbor is better, move to it
        if neighbor_fitness > current_fitness:
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
            print(f"Iteration {i}: Found better solution with fitness {current_fitness}")

    print("\nHill Climbing finished")
    return current_solution, current_fitness

In [11]:
def simulated_annealing(problem, initial_temp, final_temp, cooling_rate):
    """
    Performs a Simulated Annealing search to solve the MMKP.
    """
    num_knapsacks = problem["NUM_KNAPSACKS"]
    num_items = problem["NUM_ITEMS"]

    # Start with a valid random solution
    current_solution = generate_clean_random_solution(num_knapsacks, num_items)
    current_fitness = fitness(current_solution, problem)

    # We need to keep track of the best solution found so far
    best_solution = current_solution
    best_fitness = current_fitness

    temp = initial_temp
    print(f"Starting SA with initial fitness: {current_fitness}, Temp: {temp:.2f}")

    iteration = 0
    while temp > final_temp:
        # Create a neighbor solution by making one small change
        neighbor_solution = current_solution.copy()
        item_to_move = np.random.randint(num_items)
        new_knapsack = np.random.randint(-1, num_knapsacks)
        
        # Move the item
        neighbor_solution[:, item_to_move] = 0
        if new_knapsack != -1:
            neighbor_solution[new_knapsack, item_to_move] = 1

        neighbor_fitness = fitness(neighbor_solution, problem)
        
        # Calculate the change in fitness
        delta_fitness = neighbor_fitness - current_fitness

        # Core of the SA algorithm: The Acceptance Criterion
        if delta_fitness > 0:
            # If the neighbor is better, always accept it
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness
        else:
            # If the neighbor is worse, accept it with a certain probability
            # The probability decreases as temperature drops
            acceptance_probability = math.exp(delta_fitness / temp)
            if np.random.random() < acceptance_probability:
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness

        # Update the overall best solution found
        if current_fitness > best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness
            print(f"Iter {iteration}: New best fitness -> {best_fitness} (Temp: {temp:.2f})")
            
        # Cool the temperature
        temp *= cooling_rate
        iteration += 1

    print("\nSimulated Annealing finished.")
    return best_solution, best_fitness

## TEST PROBLEMS

In [12]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# getting problem p1
p1 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [13]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p2 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

In [14]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

p3 = get_problem(NUM_KNAPSACKS, NUM_ITEMS, NUM_DIMENSIONS, VALUES, WEIGHTS, CONSTRAINTS)

#### TEST - HILL CLIMBING

In [83]:
print("TEST - HILL CLIMBING")
print("PROBLEM 1")
best_solution, best_fitness = hill_climbing(p1, max_iterations = 20000)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

TEST - HILL CLIMBING
PROBLEM 1
Starting Hill Climbing with initial fitness: 644
Iteration 1: Found better solution with fitness 656
Iteration 2: Found better solution with fitness 699
Iteration 8: Found better solution with fitness 775
Iteration 13: Found better solution with fitness 872
Iteration 23: Found better solution with fitness 955
Iteration 67: Found better solution with fitness 1028
Iteration 68: Found better solution with fitness 1048
Iteration 106: Found better solution with fitness 1056
Iteration 271: Found better solution with fitness 1065

Hill Climbing finished
Best fitness: 1065
Best solution:
 [[0 0 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1]
 [1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0]]


In [84]:
print("PROBLEM 2")
best_solution, best_fitness = hill_climbing(p2, max_iterations = 20000)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 2
Starting Hill Climbing with initial fitness: 0

Hill Climbing finished
Best fitness: 0
Best solution:
 [[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [85]:
print("PROBLEM 3")
best_solution, best_fitness = hill_climbing(p3, max_iterations = 1000)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

PROBLEM 3
Starting Hill Climbing with initial fitness: 0

Hill Climbing finished
Best fitness: 0
Best solution:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### TEST - SIMULATED ANNILING

In [ ]:
print("TEST - SIMULATED ANNEALING")

INITIAL_TEMPERATURE = 100.0
FINAL_TEMPERATURE = 0.1
COOLING_RATE = 0.999

In [ ]:
print("PROBLEM 1")
best_solution, best_fitness = simulated_annealing(p1, INITIAL_TEMPERATURE, FINAL_TEMPERATURE, COOLING_RATE)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

In [ ]:
print("PROBLEM 2")
best_solution, best_fitness = simulated_annealing(p2, INITIAL_TEMPERATURE, FINAL_TEMPERATURE, COOLING_RATE)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")

In [ ]:
print("PROBLEM 3")
best_solution, best_fitness = simulated_annealing(p3, INITIAL_TEMPERATURE, FINAL_TEMPERATURE, COOLING_RATE)

print(f"Best fitness: {best_fitness}")
print(f"Best solution:\n {best_solution}")